# Preprocessing ENTSO-E Energy Load Data for the Netherlands
### Data Source
The energy load data can be found at: https://www.entsoe.eu/data/power-stats/

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import warnings
import math
from math import sqrt
import time
warnings.filterwarnings("ignore")


## 1. Load consumption data | Load consumption in MW

Netherlands energy load from 1-1-2006 00:00:00 - 31-12-2015 23:00:00

In [2]:
# Energy load for 2006-2015
df1 = pd.read_excel('load_2006_2015.xlsx')

load_2006_2015_NL = df1[df1['Country'] == 'NL']
load_2006_2015_NL.head()

,Country,Year,Month,Day,Coverage ratio,0,1,2,3,4,...,14,15,16,17,18,19,20,21,22,23
83568,NL,2006,1,1,100.0,10215.0,9979.0,9460.0,8833.0,8525.0,...,11305.0,11202.0,11553.0,12691.0,13134.0,13094.0,12719.0,12130.0,11464.0,10482.0
83569,NL,2006,1,2,100.0,9590.0,9017.0,8758.0,8580.0,8621.0,...,15464.0,15202.0,15292.0,16301.0,15926.0,15583.0,14725.0,13936.0,12905.0,12165.0
83570,NL,2006,1,3,100.0,11064.0,10145.0,9717.0,9573.0,9634.0,...,15600.0,15435.0,15708.0,16592.0,15995.0,15563.0,14732.0,13958.0,12922.0,12184.0
83571,NL,2006,1,4,100.0,11088.0,10221.0,9859.0,9700.0,9730.0,...,15896.0,15799.0,15973.0,16581.0,15907.0,15465.0,14568.0,13837.0,12773.0,12076.0
83572,NL,2006,1,5,100.0,10925.0,10029.0,9612.0,9386.0,9501.0,...,16042.0,16003.0,16233.0,16514.0,15921.0,15569.0,14851.0,13926.0,12833.0,12113.0


In [3]:
# Convert column names to strings to use .isdigit() safely
load_2006_2015_NL.columns = load_2006_2015_NL.columns.astype(str)
    
# Identify columns corresponding to hours by checking if column names are numeric
hour_columns = [col for col in load_2006_2015_NL.columns if col.isdigit()]

# Reshape data to long format, focusing only on necessary columns
load_2006_2015_NL = pd.melt(
    load_2006_2015_NL,
    id_vars=["Year", "Month", "Day"],  # Only necessary columns
    value_vars=hour_columns,  # Limit to hour columns only
    var_name="Hour",
    value_name="loadConsumption"
)

# Convert hour column to integer for datetime operations
load_2006_2015_NL['Hour'] = load_2006_2015_NL['Hour'].astype(int)

# Convert to datetime formats for date and datetime columns
load_2006_2015_NL['date'] = pd.to_datetime(load_2006_2015_NL[['Year', 'Month', 'Day']])
load_2006_2015_NL['datetime'] = pd.to_datetime(load_2006_2015_NL['date']) + pd.to_timedelta(load_2006_2015_NL['Hour'] - 1, unit='h')

# Drop unnecessary columns for cleaner output
load_2006_2015_NL.drop(columns=['Month', 'Day'], inplace=True)

load_2006_2015_NL.head()

,Year,Hour,loadConsumption,date,datetime
0,2006,0,10215.0,2006-01-01,2005-12-31 23:00:00
1,2006,0,9590.0,2006-01-02,2006-01-01 23:00:00
2,2006,0,11064.0,2006-01-03,2006-01-02 23:00:00
3,2006,0,11088.0,2006-01-04,2006-01-03 23:00:00
4,2006,0,10925.0,2006-01-05,2006-01-04 23:00:00


Netherlands energy load from 31-12-2015 00:00:00 - 31-12-2017 23:00:00

In [4]:
df2 = pd.read_excel('load_2015_2019.xlsx', sheet_name='2015-2017')
load_2015_2017_NL = df2[df2['CountryCode'] == 'NL']
load_2015_2017_NL

,MeasureItem,DateUTC,DateShort,TimeFrom,TimeTo,CountryCode,Cov_ratio,Value,Value_ScaleTo100
8790,Monthly Hourly Load Values,2015-12-31 23:00:00,2015-12-31,23:00:00,00:00:00,NL,100,11373.0,11373.0
8824,Monthly Hourly Load Values,2016-01-01 00:00:00,2016-01-01,00:00:00,01:00:00,NL,100,11102.0,11102.0
8859,Monthly Hourly Load Values,2016-01-01 01:00:00,2016-01-01,01:00:00,02:00:00,NL,100,10657.0,10657.0
8894,Monthly Hourly Load Values,2016-01-01 02:00:00,2016-01-01,02:00:00,03:00:00,NL,100,10238.0,10238.0
8929,Monthly Hourly Load Values,2016-01-01 03:00:00,2016-01-01,03:00:00,04:00:00,NL,100,10018.0,10018.0
...,...,...,...,...,...,...,...,...,...
631446,Monthly Hourly Load Values,2017-12-31 19:00:00,2017-12-31,19:00:00,20:00:00,NL,100,12139.0,12139.0
631482,Monthly Hourly Load Values,2017-12-31 20:00:00,2017-12-31,20:00:00,21:00:00,NL,100,11207.0,11207.0
631518,Monthly Hourly Load Values,2017-12-31 21:00:00,2017-12-31,21:00:00,22:00:00,NL,100,11175.0,11175.0
631554,Monthly Hourly Load Values,2017-12-31 22:00:00,2017-12-31,22:00:00,23:00:00,NL,100,13121.0,13121.0


In [5]:
df2 = pd.read_excel('load_2015_2019.xlsx', sheet_name='2018-2019')
load_2018_2019_NL = df2[df2['CountryCode'] == 'NL']
load_2018_2019_NL

,MeasureItem,DateUTC,DateShort,TimeFrom,TimeTo,CountryCode,Cov_ratio,Value,Value_ScaleTo100
26,Monthly Hourly Load Values,2018-01-01 00:00:00,2018-01-01,00:00:00,01:00:00,NL,100,11255.0,11255.0
62,Monthly Hourly Load Values,2018-01-01 01:00:00,2018-01-01,01:00:00,02:00:00,NL,100,10956.0,10956.0
98,Monthly Hourly Load Values,2018-01-01 02:00:00,2018-01-01,02:00:00,03:00:00,NL,100,10647.0,10647.0
134,Monthly Hourly Load Values,2018-01-01 03:00:00,2018-01-01,03:00:00,04:00:00,NL,100,10431.0,10431.0
170,Monthly Hourly Load Values,2018-01-01 04:00:00,2018-01-01,04:00:00,05:00:00,NL,100,10431.0,10431.0
...,...,...,...,...,...,...,...,...,...
398430,Monthly Hourly Load Values,2019-04-30 17:00:00,2019-04-30,17:00:00,18:00:00,NL,100,13910.0,13910.0
398458,Monthly Hourly Load Values,2019-04-30 18:00:00,2019-04-30,18:00:00,19:00:00,NL,100,13773.0,13773.0
398486,Monthly Hourly Load Values,2019-04-30 19:00:00,2019-04-30,19:00:00,20:00:00,NL,100,13652.0,13652.0
398514,Monthly Hourly Load Values,2019-04-30 20:00:00,2019-04-30,20:00:00,21:00:00,NL,100,13048.0,13048.0


In [6]:
# Concatenate the two datasets for 2015-2019
load_2015_2019_NL = pd.concat([load_2015_2017_NL, load_2018_2019_NL])

# Convert DateShort to datetime format
load_2015_2019_NL['DateShort'] = pd.to_datetime(load_2015_2019_NL['DateShort'], dayfirst=True)

# Filter rows based on the date condition
load_2015_2019_NL = load_2015_2019_NL.loc[load_2015_2019_NL['DateShort'] >= '2016-01-01']

# Reset index after filtering
load_2015_2019_NL.reset_index(drop=True, inplace=True)

In [7]:
load_2015_2019_NL

,MeasureItem,DateUTC,DateShort,TimeFrom,TimeTo,CountryCode,Cov_ratio,Value,Value_ScaleTo100
0,Monthly Hourly Load Values,2016-01-01 00:00:00,2016-01-01,00:00:00,01:00:00,NL,100,11102.0,11102.0
1,Monthly Hourly Load Values,2016-01-01 01:00:00,2016-01-01,01:00:00,02:00:00,NL,100,10657.0,10657.0
2,Monthly Hourly Load Values,2016-01-01 02:00:00,2016-01-01,02:00:00,03:00:00,NL,100,10238.0,10238.0
3,Monthly Hourly Load Values,2016-01-01 03:00:00,2016-01-01,03:00:00,04:00:00,NL,100,10018.0,10018.0
4,Monthly Hourly Load Values,2016-01-01 04:00:00,2016-01-01,04:00:00,05:00:00,NL,100,10017.0,10017.0
...,...,...,...,...,...,...,...,...,...
29177,Monthly Hourly Load Values,2019-04-30 17:00:00,2019-04-30,17:00:00,18:00:00,NL,100,13910.0,13910.0
29178,Monthly Hourly Load Values,2019-04-30 18:00:00,2019-04-30,18:00:00,19:00:00,NL,100,13773.0,13773.0
29179,Monthly Hourly Load Values,2019-04-30 19:00:00,2019-04-30,19:00:00,20:00:00,NL,100,13652.0,13652.0
29180,Monthly Hourly Load Values,2019-04-30 20:00:00,2019-04-30,20:00:00,21:00:00,NL,100,13048.0,13048.0


In [8]:
# Filter relevant columns
load_2015_2019_NL = load_2015_2019_NL[['DateUTC', 'DateShort', 'Value']].copy()

# Convert date columns to datetime format
load_2015_2019_NL['date'] = pd.to_datetime(load_2015_2019_NL['DateShort'], dayfirst=True, format='%d/%m/%Y')
load_2015_2019_NL['datetime'] = pd.to_datetime(load_2015_2019_NL['DateUTC'], format='%d/%m/%Y %H:%M')

# Extract hour, year, and week directly from the datetime columns
load_2015_2019_NL['Hour'] = load_2015_2019_NL['datetime'].dt.hour + 1  # 1–24 range
load_2015_2019_NL['Year'] = load_2015_2019_NL['date'].dt.year

# Rename 'Value' to 'loadConsumption' 
load_2015_2019_NL.rename(columns={'Value': 'loadConsumption'}, inplace=True)

# Drop intermediate columns that are no longer needed
load_2015_2019_NL.drop(columns=['DateShort', 'DateUTC'], inplace=True)

In [9]:
load_2015_2019_NL

,loadConsumption,date,datetime,Hour,Year
0,11102.0,2016-01-01,2016-01-01 00:00:00,1,2016
1,10657.0,2016-01-01,2016-01-01 01:00:00,2,2016
2,10238.0,2016-01-01,2016-01-01 02:00:00,3,2016
3,10018.0,2016-01-01,2016-01-01 03:00:00,4,2016
4,10017.0,2016-01-01,2016-01-01 04:00:00,5,2016
...,...,...,...,...,...
29177,13910.0,2019-04-30,2019-04-30 17:00:00,18,2019
29178,13773.0,2019-04-30,2019-04-30 18:00:00,19,2019
29179,13652.0,2019-04-30,2019-04-30 19:00:00,20,2019
29180,13048.0,2019-04-30,2019-04-30 20:00:00,21,2019


In [10]:
load_2006_2019 = pd.concat([load_2006_2015_NL, load_2015_2019_NL])
load_2006_2019.reset_index(inplace=True, drop=True)

In [11]:
load_2006_2019

,Year,Hour,loadConsumption,date,datetime
0,2006,0,10215.0,2006-01-01,2005-12-31 23:00:00
1,2006,0,9590.0,2006-01-02,2006-01-01 23:00:00
2,2006,0,11064.0,2006-01-03,2006-01-02 23:00:00
3,2006,0,11088.0,2006-01-04,2006-01-03 23:00:00
4,2006,0,10925.0,2006-01-05,2006-01-04 23:00:00
...,...,...,...,...,...
116825,2019,18,13910.0,2019-04-30,2019-04-30 17:00:00
116826,2019,19,13773.0,2019-04-30,2019-04-30 18:00:00
116827,2019,20,13652.0,2019-04-30,2019-04-30 19:00:00
116828,2019,21,13048.0,2019-04-30,2019-04-30 20:00:00


In [13]:
files = ['load_2019.xlsx','load_2020.xlsx', 'load_2021.xlsx', 'load_2022.xlsx', 'load_2023.xlsx', 'load_2024.xlsx', 'load_2025.xlsx']

filtered_data = []

for file in files:
    df = pd.read_excel(file)  
    df_nl = df[df['CountryCode'] == 'NL']  # Filter rows for the Netherlands
    filtered_data.append(df_nl)  

# Combine all filtered data into a single DataFrame
combined_2019_2025 = pd.concat(filtered_data, ignore_index=True)

combined_2019_2025.head()

,MeasureItem,DateUTC,DateShort,TimeFrom,TimeTo,CountryCode,Cov_ratio,Value,Value_ScaleTo100,CreateDate,UpdateDate
0,Monthly Hourly Load Values,2019-01-01 00:00:00,2019-01-01,1970-01-01 00:00:00,1970-01-01 01:00:00,NL,100,11193.6350,11193.6350,2024-05-29 11:37:27.260,2024-05-29 11:37:27.260
1,Monthly Hourly Load Values,2019-01-01 01:00:00,2019-01-01,1970-01-01 01:00:00,1970-01-01 02:00:00,NL,100,10933.3600,10933.3600,2024-05-29 11:37:27.260,2024-05-29 11:37:27.260
2,Monthly Hourly Load Values,2019-01-01 02:00:00,2019-01-01,1970-01-01 02:00:00,1970-01-01 03:00:00,NL,100,10666.5800,10666.5800,2024-05-29 11:37:27.260,2024-05-29 11:37:27.260
3,Monthly Hourly Load Values,2019-01-01 03:00:00,2019-01-01,1970-01-01 03:00:00,1970-01-01 04:00:00,NL,100,10413.3575,10413.3575,2024-05-29 11:37:27.260,2024-05-29 11:37:27.260
4,Monthly Hourly Load Values,2019-01-01 04:00:00,2019-01-01,1970-01-01 04:00:00,1970-01-01 05:00:00,NL,100,10440.7925,10440.7925,2024-05-29 11:37:27.260,2024-05-29 11:37:27.260


In [14]:
missing_percentage = combined_2019_2025.isnull().mean() * 100

# Display the missing percentage for each column
print(missing_percentage)


MeasureItem          0.000000
DateUTC              0.000000
DateShort            0.000000
TimeFrom             0.000000
TimeTo               0.000000
CountryCode          0.000000
Cov_ratio            0.000000
Value                0.000000
Value_ScaleTo100     0.000000
CreateDate          14.807302
UpdateDate          14.807302
dtype: float64


In [15]:
# Drop columns with missing values
columns_to_drop = ['CreateDate', 'UpdateDate']  

combined_2019_2025 = combined_2019_2025.drop(columns=columns_to_drop)

In [16]:
combined_2019_2025

,MeasureItem,DateUTC,DateShort,TimeFrom,TimeTo,CountryCode,Cov_ratio,Value,Value_ScaleTo100
0,Monthly Hourly Load Values,2019-01-01 00:00:00,2019-01-01,1970-01-01 00:00:00,1970-01-01 01:00:00,NL,100,11193.6350,11193.6350
1,Monthly Hourly Load Values,2019-01-01 01:00:00,2019-01-01,1970-01-01 01:00:00,1970-01-01 02:00:00,NL,100,10933.3600,10933.3600
2,Monthly Hourly Load Values,2019-01-01 02:00:00,2019-01-01,1970-01-01 02:00:00,1970-01-01 03:00:00,NL,100,10666.5800,10666.5800
3,Monthly Hourly Load Values,2019-01-01 03:00:00,2019-01-01,1970-01-01 03:00:00,1970-01-01 04:00:00,NL,100,10413.3575,10413.3575
4,Monthly Hourly Load Values,2019-01-01 04:00:00,2019-01-01,1970-01-01 04:00:00,1970-01-01 05:00:00,NL,100,10440.7925,10440.7925
...,...,...,...,...,...,...,...,...,...
59155,Monthly Hourly Load Values,2025-09-30 09:00:00,2025-09-30,1970-01-01 09:00:00,1970-01-01 10:00:00,NL,100,13711.0100,13711.0100
59156,Monthly Hourly Load Values,2025-08-02 14:00:00,2025-08-02,1970-01-01 14:00:00,1970-01-01 15:00:00,NL,100,11190.8275,11190.8275
59157,Monthly Hourly Load Values,2025-01-06 23:00:00,2025-01-06,1970-01-01 23:00:00,1970-01-01 00:00:00,NL,100,13372.5300,13372.5300
59158,Monthly Hourly Load Values,2025-09-21 13:00:00,2025-09-21,1970-01-01 13:00:00,1970-01-01 14:00:00,NL,100,11628.1600,11628.1600


In [17]:
combined_2019_2025 = combined_2019_2025[['DateUTC', 'DateShort', 'Value']].copy()

# Convert date columns to datetime format
combined_2019_2025['date'] = pd.to_datetime(combined_2019_2025['DateShort'], dayfirst=True, format='%Y-%m-%d')  
combined_2019_2025['datetime'] = pd.to_datetime(combined_2019_2025['DateUTC'], format='%Y-%m-%d %H:%M')

# Extract hour, year, and other necessary components
combined_2019_2025['Hour'] = combined_2019_2025['datetime'].dt.hour + 1  # Adjust for 1–24 hour range
combined_2019_2025['Year'] = combined_2019_2025['date'].dt.year

#Rename 'Value' column to match the old dataset naming convention
combined_2019_2025.rename(columns={'Value': 'loadConsumption'}, inplace=True)

combined_2019_2025.drop(columns=['DateShort', 'DateUTC'], inplace=True)

#Check the transformed dataset
combined_2019_2025.head()

,loadConsumption,date,datetime,Hour,Year
0,11193.6350,2019-01-01,2019-01-01 00:00:00,1,2019
1,10933.3600,2019-01-01,2019-01-01 01:00:00,2,2019
2,10666.5800,2019-01-01,2019-01-01 02:00:00,3,2019
3,10413.3575,2019-01-01,2019-01-01 03:00:00,4,2019
4,10440.7925,2019-01-01,2019-01-01 04:00:00,5,2019


In [22]:
total_data_2006_2025= pd.concat([combined_2019_2025, load_2006_2019], axis=0).sort_index()
total_data_2006_2025

,loadConsumption,date,datetime,Hour,Year
0,11193.635,2019-01-01,2019-01-01 00:00:00,1,2019
0,10215.000,2006-01-01,2005-12-31 23:00:00,0,2006
1,9590.000,2006-01-02,2006-01-01 23:00:00,0,2006
1,10933.360,2019-01-01,2019-01-01 01:00:00,2,2019
2,10666.580,2019-01-01,2019-01-01 02:00:00,3,2019
...,...,...,...,...,...
116825,13910.000,2019-04-30,2019-04-30 17:00:00,18,2019
116826,13773.000,2019-04-30,2019-04-30 18:00:00,19,2019
116827,13652.000,2019-04-30,2019-04-30 19:00:00,20,2019
116828,13048.000,2019-04-30,2019-04-30 20:00:00,21,2019


In [23]:
total_data_2006_2025['datetime'] = pd.to_datetime(total_data_2006_2025['datetime'], errors='coerce')

In [24]:
print(total_data_2006_2025['datetime'].isnull().sum())

0


In [25]:
total_data_2006_2025 = total_data_2006_2025.sort_values(by='datetime').reset_index(drop=True)
total_data_2006_2025

,loadConsumption,date,datetime,Hour,Year
0,10215.0000,2006-01-01,2005-12-31 23:00:00,0,2006
1,9979.0000,2006-01-01,2006-01-01 00:00:00,1,2006
2,9460.0000,2006-01-01,2006-01-01 01:00:00,2,2006
3,8833.0000,2006-01-01,2006-01-01 02:00:00,3,2006
4,8525.0000,2006-01-01,2006-01-01 03:00:00,4,2006
...,...,...,...,...,...
175985,14224.9775,2025-09-30,2025-09-30 19:00:00,20,2025
175986,13410.5000,2025-09-30,2025-09-30 20:00:00,21,2025
175987,12497.6325,2025-09-30,2025-09-30 21:00:00,22,2025
175988,11788.7800,2025-09-30,2025-09-30 22:00:00,23,2025


In [26]:
complete_dates = pd.date_range(start=total_data_2006_2025.index.min(),
                               end=total_data_2006_2025.index.max(),
                               freq='H')  # Adjust frequency if needed

In [27]:
missing_dates = complete_dates.difference(total_data_2006_2025.index)
print(f"Number of missing dates: {len(missing_dates)}")
print("Missing dates:")
print(missing_dates)


Number of missing dates: 1
Missing dates:
DatetimeIndex(['1970-01-01 00:00:00'], dtype='datetime64[ns]', freq='h')


In [28]:
total_data_2006_2025 = total_data_2006_2025[total_data_2006_2025.index != '1970-01-01']

In [29]:
print('1970-01-01' in total_data_2006_2023.index)


False


In [30]:
missing_data = total_data_2006_2025.isnull().sum()
print(missing_data)


loadConsumption    10
date                0
datetime            0
Hour                0
Year                0
dtype: int64


In [31]:
total_data_2006_2025[total_data_2006_2025['loadConsumption'].isnull()]

,loadConsumption,date,datetime,Hour,Year
2018,NaN,2006-03-26,2006-03-26 01:00:00,2,2006
10754,NaN,2007-03-25,2007-03-25 01:00:00,2,2007
19658,NaN,2008-03-30,2008-03-30 01:00:00,2,2008
28394,NaN,2009-03-29,2009-03-29 01:00:00,2,2009
37130,NaN,2010-03-28,2010-03-28 01:00:00,2,2010
45866,NaN,2011-03-27,2011-03-27 01:00:00,2,2011
54602,NaN,2012-03-25,2012-03-25 01:00:00,2,2012
63506,NaN,2013-03-31,2013-03-31 01:00:00,2,2013
72242,NaN,2014-03-30,2014-03-30 01:00:00,2,2014
80978,NaN,2015-03-29,2015-03-29 01:00:00,2,2015


In [32]:
def fill_with_average(series):

    return series.fillna((series.shift() + series.shift(-1)) / 2)

# Apply to a specific to loadConsumption
total_data_2006_2025['loadConsumption'] = fill_with_average(total_data_2006_2025['loadConsumption'])


In [33]:
missing_data = total_data_2006_2025.isnull().sum()
print(missing_data)

loadConsumption    0
date               0
datetime           0
Hour               0
Year               0
dtype: int64


In [34]:
data_2019 = total_data_2006_2025[total_data_2006_2025['Year'] == 2019]

In [35]:
duplicates_date = data_2019[data_2019.duplicated(subset=['datetime'], keep=False)]
print("Duplicates based on 'datetime':")
print(duplicates_date)


Duplicates based on 'datetime':
        loadConsumption       date            datetime  Hour  Year
113952       11193.6350 2019-01-01 2019-01-01 00:00:00     1  2019
113953       11466.0000 2019-01-01 2019-01-01 00:00:00     1  2019
113954       11207.0000 2019-01-01 2019-01-01 01:00:00     2  2019
113955       10933.3600 2019-01-01 2019-01-01 01:00:00     2  2019
113956       10666.5800 2019-01-01 2019-01-01 02:00:00     3  2019
...                 ...        ...                 ...   ...   ...
119703       13661.3600 2019-04-30 2019-04-30 19:00:00    20  2019
119704       13059.5350 2019-04-30 2019-04-30 20:00:00    21  2019
119705       13048.0000 2019-04-30 2019-04-30 20:00:00    21  2019
119706       12408.0000 2019-04-30 2019-04-30 21:00:00    22  2019
119707       12420.8925 2019-04-30 2019-04-30 21:00:00    22  2019

[5755 rows x 5 columns]


In [36]:
data_2019_cleaned = data_2019.drop_duplicates(subset=['datetime'], keep='last')

In [37]:
total_data_2006_2025 = total_data_2006_2025[total_data_2006_2025['Year'] != 2019]

# Append the cleaned 2019 data
total_data_2006_2025 = pd.concat([total_data_2006_2025, data_2019_cleaned])

# Sort and reset index
total_data_2006_2025 = total_data_2006_2025.sort_values(by='datetime').reset_index(drop=True)

In [38]:
duplicates_date_total = total_data_2006_2025[total_data_2006_2025.duplicated(subset=['datetime'], keep=False)]
print("Duplicates based on 'datetime':")
print(duplicates_date_total)

Duplicates based on 'datetime':
        loadConsumption       date            datetime  Hour  Year
124826        9700.3175 2020-03-29 2020-03-29 03:00:00     4  2020
124827        9800.4350 2020-03-29 2020-03-29 03:00:00     4  2020
133562        9888.5300 2021-03-28 2021-03-28 03:00:00     4  2021
133563        9812.5000 2021-03-28 2021-03-28 03:00:00     4  2021
142298        9285.1450 2022-03-27 2022-03-27 03:00:00     4  2022
142299        9292.0550 2022-03-27 2022-03-27 03:00:00     4  2022
151034        9313.6250 2023-03-26 2023-03-26 03:00:00     4  2023
151035        9338.1725 2023-03-26 2023-03-26 03:00:00     4  2023
159938       10172.6650 2024-03-31 2024-03-31 03:00:00     4  2024
159939       10197.1875 2024-03-31 2024-03-31 03:00:00     4  2024


In [39]:
total_data_2006_2025 = total_data_2006_2025.drop_duplicates(subset=['datetime'], keep='last')


In [40]:
total_data_2006_2025.to_csv('data_2006_2025.csv', index=False)
